In [2]:
import keras_nlp
import keras
import tensorflow as tf
import os
import shutil
import pickle

os.environ["KERAS_BACKEND"] = "tensorflow"
keras.utils.set_random_seed(42)

In [4]:
BATCH_SIZE = 64
EPOCHS = 3
MAX_SEQUENCE_LENGTH = 512
VOCAB_SIZE = 15000

EMBED_DIM = 128
INTERMEDIATE_DIM = 512

In [6]:
os.getcwd()

'/Users/natalies_macbook/Documents/GitHub/INSA/EU'

In [65]:
import os

# Define the input and output directories
input_dir = '/Users/natalies_macbook/Downloads/Korpus/from_html'
output_dir = '/Users/natalies_macbook/Downloads/Korpus/unseen'

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Process each HTML file in the input directory
for filename in os.listdir(input_dir):
    if filename.endswith('.html'):
        input_path = os.path.join(input_dir, filename)
        output_filename = os.path.splitext(filename)[0] + '.txt'
        output_path = os.path.join(output_dir, output_filename)

        # Read the HTML file
        with open(input_path, 'r', encoding='utf-8') as file:
            html_content = file.read()

        # Save the HTML content to a new file
        with open(output_path, 'w', encoding='utf-8') as file:
            file.write(html_content)

        print(f'Copied {input_path} to {output_path}')


Copied /Users/natalies_macbook/Downloads/Korpus/from_html/document_1240.html to /Users/natalies_macbook/Downloads/Korpus/unseen/document_1240.txt
Copied /Users/natalies_macbook/Downloads/Korpus/from_html/document_3787.html to /Users/natalies_macbook/Downloads/Korpus/unseen/document_3787.txt
Copied /Users/natalies_macbook/Downloads/Korpus/from_html/document_2895.html to /Users/natalies_macbook/Downloads/Korpus/unseen/document_2895.txt
Copied /Users/natalies_macbook/Downloads/Korpus/from_html/document_1610.html to /Users/natalies_macbook/Downloads/Korpus/unseen/document_1610.txt
Copied /Users/natalies_macbook/Downloads/Korpus/from_html/document_2196.html to /Users/natalies_macbook/Downloads/Korpus/unseen/document_2196.txt
Copied /Users/natalies_macbook/Downloads/Korpus/from_html/document_2529.html to /Users/natalies_macbook/Downloads/Korpus/unseen/document_2529.txt
Copied /Users/natalies_macbook/Downloads/Korpus/from_html/document_2179.html to /Users/natalies_macbook/Downloads/Korpus/uns

In [103]:
import os
import fitz  # PyMuPDF

# Define the input and output directories
input_dir = '/Users/natalies_macbook/Downloads/Korpus/from_pdf'
output_dir = '/Users/natalies_macbook/Downloads/Korpus/unseen_pdf'

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Process each PDF file in the input directory
for filename in os.listdir(input_dir):
    if filename.endswith('.pdf'):
        input_path = os.path.join(input_dir, filename)
        output_filename = os.path.splitext(filename)[0] + '.txt'
        output_path = os.path.join(output_dir, output_filename)

        # Open the PDF file
        with fitz.open(input_path) as pdf_document:
            # Save the PDF content to a new file
            pdf_document.save(output_path)

        print(f'Copied {input_path} to {output_path}')

Copied /Users/natalies_macbook/Downloads/Korpus/from_pdf/document_1111.pdf to /Users/natalies_macbook/Downloads/Korpus/unseen_pdf/document_1111.txt
Copied /Users/natalies_macbook/Downloads/Korpus/from_pdf/document_2791.pdf to /Users/natalies_macbook/Downloads/Korpus/unseen_pdf/document_2791.txt
Copied /Users/natalies_macbook/Downloads/Korpus/from_pdf/document_2551.pdf to /Users/natalies_macbook/Downloads/Korpus/unseen_pdf/document_2551.txt
Copied /Users/natalies_macbook/Downloads/Korpus/from_pdf/document_666.pdf to /Users/natalies_macbook/Downloads/Korpus/unseen_pdf/document_666.txt
Copied /Users/natalies_macbook/Downloads/Korpus/from_pdf/document_2155.pdf to /Users/natalies_macbook/Downloads/Korpus/unseen_pdf/document_2155.txt
Copied /Users/natalies_macbook/Downloads/Korpus/from_pdf/document_328.pdf to /Users/natalies_macbook/Downloads/Korpus/unseen_pdf/document_328.txt
Copied /Users/natalies_macbook/Downloads/Korpus/from_pdf/document_2209.pdf to /Users/natalies_macbook/Downloads/Korp

FileDataError: Failed to open file '/Users/natalies_macbook/Downloads/Korpus/from_pdf/document_3961.pdf'.

In [105]:
PATH_UNSEEN_DATA = "/Users/natalies_macbook/Downloads/Korpus/unseen_pdf"
RESULTS_POS_DIR = "/Users/natalies_macbook/Documents/"
RESULTS_NEG_DIR = "/Users/natalies_macbook/Documents/"

In [107]:
# load our pre-trained classifier
insa_classifier = keras.models.load_model('insa_20240712.keras')

In [109]:
# load vocabulary
with open('INSA_vocab.pickle', 'rb') as in_file:
    vocab = pickle.load(in_file)


In [110]:
# create a tokenizer with our vocabulary
tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=vocab,
    lowercase=False,
    sequence_length=MAX_SEQUENCE_LENGTH,
)

In [111]:
# create a dataset from previously unseen data
# path is specific to file location, points to a folder with .txt files that we want to classify
unseen_ds = keras.utils.text_dataset_from_directory(
    PATH_UNSEEN_DATA,
    batch_size=BATCH_SIZE,
    seed=42,
    labels=None,
)

Found 55 files.


In [115]:
# everything has to be in lowercase
unseen_ds = unseen_ds.map(lambda x: (tf.strings.lower(x)))

In [116]:
unseen_ds

<_MapDataset element_spec=TensorSpec(shape=(None,), dtype=tf.string, name=None)>

In [117]:
# format our dataset
# this takes care of files that are longer than our 512 cut-off batchsize (all information is retained)
def format_dataset(sentence):
    sentence = tokenizer(sentence)
    return ({"input_ids": sentence})


def make_dataset(dataset):
    dataset = dataset.map(format_dataset, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset.shuffle(512).prefetch(16).cache()

unseen_ds = make_dataset(unseen_ds)

In [119]:
# prediction on unseen data
unseen_results = insa_classifier.predict(unseen_ds)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 469ms/step


In [121]:
unseen_results

array([[0.338348  ],
       [0.06726368],
       [0.164464  ],
       [0.2902687 ],
       [0.18847018],
       [0.13917893],
       [0.13273193],
       [0.0101713 ],
       [0.01139297],
       [0.14933771],
       [0.05718992],
       [0.13550866],
       [0.567239  ],
       [0.5854252 ],
       [0.5228244 ],
       [0.23629972],
       [0.10804443],
       [0.19930452],
       [0.19463672],
       [0.02421895],
       [0.07169615],
       [0.0652279 ],
       [0.21073174],
       [0.04834099],
       [0.06496221],
       [0.08762537],
       [0.02807691],
       [0.06701747],
       [0.16652487],
       [0.11073919],
       [0.19864346],
       [0.10804443],
       [0.21434526],
       [0.08695806],
       [0.14549166],
       [0.01841864],
       [0.41038582],
       [0.10804443],
       [0.09555754],
       [0.14832425],
       [0.10804443],
       [0.24696289],
       [0.22450407],
       [0.02455304],
       [0.04000509],
       [0.09292144],
       [0.04912705],
       [0.442

In [127]:
# this should return True, if there are only .txt files in our directory, and every file was classified
len(os.listdir(PATH_UNSEEN_DATA)) == len(unseen_results)

True

In [129]:
# this is a list of all files in the unseen directory
all_unseen_files = os.listdir(PATH_UNSEEN_DATA)

# loop over our results
# the order of those elements matches the list above
list_1 = []
for i in range(0, len(unseen_results)):
    result_score = unseen_results[i]
    current_file_name = all_unseen_files[i]
    dict_1 = {current_file_name:result_score}
    list_1.append(dict_1)
    if result_score[0] > 0.2:
        # this means our file is relevant
        shutil.move(os.path.join(PATH_UNSEEN_DATA, current_file_name), os.path.join(RESULTS_POS_DIR, current_file_name))
    else:
        # file is not relevant
        shutil.move(os.path.join(PATH_UNSEEN_DATA, current_file_name), os.path.join(RESULTS_NEG_DIR, current_file_name))

The first run gave 2736 irrelevant and 1166 relevant documents. 
However, after inspecting just a sample of 5 irrelevant texts, it became clear, that a lot of them were falsely classified. I could be that I did not keep the html code and thats why it falsely classified them, ill retry with keeping the html code and check for the numbers. After running it again but keeping the html code 1814 irrelevant documents and 2183 relevant documents were classified. A look into some of the irrelevant classified documents shows, that there is still some missclassification going on. 
Therefore the process was repeated and the threshold set to 0.2. After that, 1434 irrelevant and 2498 relevant documents were classified.


In [131]:
import pickle
with open("threshold_check2", "wb") as fp:   #Pickling
    pickle.dump(list_1, fp)